In [13]:
!pip install apyori

In [29]:
from apyori import apriori
import numpy as np
import pandas as pd

In [31]:
df = pd.read_csv("Groceries_dataset.csv", parse_dates=["Date"], dayfirst=True)
diff_prod = df["itemDescription"].unique()
print("The total products in the set are: {}".format(len(diff_prod)))

one_hot = pd.get_dummies(df["itemDescription"])
df.drop("itemDescription", axis=1, inplace=True)
df = df.join(one_hot)

records = df.groupby(["Member_number", "Date"])[diff_prod].agg("sum").reset_index()[diff_prod]
records = records.astype(object)

def get_product(row):
    for product in diff_prod:
        if row[product] > 0:
            row[product] = str(product)
    return row

records = records.apply(get_product, axis=1)
print("The total transactions are: {}".format(len(records)))
print(f"\n")

item_record = records.values
transactions = [sub[sub != 0].tolist() for sub in item_record if sub[sub != 0].tolist()]

rules = apriori(transactions, min_support=0.00030, min_confidence=0.04, min_lift=3, min_length=2, target="rules")
association = list(rules)

for i, item in enumerate(association, start=1):
    group = item[0]
    items = list(group)
     
    if len(items) >= 2:
        print(f"{i}) Rule: {items[0]} -> {items[1]}")
        print(f" Therefore with a support of: {item[1]}")
        print(f" A confidence level of: {item[2][0][2]}")
        print(f" And a lift(likelyhood) of: {item[2][0][3]}")
        print(f" It is likely that people who purchase {items[0]} have a high likelyhood of purchasing:{items[1]}\n")

The total products in the set are: 167
The total transactions are: 14963


1) Rule: frozen fish -> specialty chocolate
 Therefore with a support of: 0.0003341575887188398
 A confidence level of: 0.049019607843137254
 And a lift(likelyhood) of: 3.0689556157190907
 It is likely that people who purchase frozen fish have a high likelyhood of purchasing:specialty chocolate

2) Rule: liver loaf -> fruit/vegetable juice
 Therefore with a support of: 0.00040098910646260775
 A confidence level of: 0.12
 And a lift(likelyhood) of: 3.5276227897838903
 It is likely that people who purchase liver loaf have a high likelyhood of purchasing:fruit/vegetable juice

3) Rule: pickled vegetables -> ham
 Therefore with a support of: 0.0005346521419501437
 A confidence level of: 0.05970149253731344
 And a lift(likelyhood) of: 3.4895055970149254
 It is likely that people who purchase pickled vegetables have a high likelyhood of purchasing:ham

4) Rule: meat -> roll products 
 Therefore with a support of: 0.00